In [35]:
import pandas as pd
import numpy as np
import math


In [ ]:
x_mat = np.matrix([
    [0, 0, 1, 1, 1, 0, 1, 1, 1, 0], 
    [1, 3, 2, 1, 2 ,1, 1, 1, 3, 2], 
    [3, 1, 2, 3, 3, 2, 2, 1, 1, 1], 
])
y_mat = np.matrix([
    -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, 
])

In [47]:
# 单层树桩分类器
def stump_classify(mat, dim, thresh_val, ascending):
    result = np.ones((1, mat.shape[1]))
    x = mat[dim]
    for i in range(len(x)):
        if ascending is True:
            result[mat[dim, :] <= thresh_val] = -1
        else:
            result[mat[dim, :] > thresh_val] = -1
    return result

# 找到最优树桩（允许预测结果 全部取1和全部取-1）
def build_stump(x_mat, y_mat, w):
    m, n = x_mat.shape
    best_stump = []
    best_acc = -1
    best_result = np.matrix([0])
    w_sum = np.sum(w)
    for dim in range(m):
        unique_vals = sorted(set(np.array(x_mat)[dim]))
        for val_floor, val_ceil in zip([unique_vals[0] - 0.1] + unique_vals, unique_vals + [unique_vals[0] + 0.1]): # 这里不能全部取负
            ascending = True
            thresh_val = (val_floor + val_ceil) / 2
            pred_result = stump_classify(x_mat, dim, thresh_val, ascending=True)
            accuracy = np.sum(np.multiply((pred_result == y_mat), w))
            if accuracy < w_sum / 2:
                ascending = False
                accuracy = w_sum - accuracy
                pred_result = - pred_result
            if accuracy > best_acc:
                best_stump = [dim, thresh_val, ascending]
                best_acc = accuracy
                best_result = pred_result
    return best_stump, best_acc, best_result

# 计算矩阵之和
def sum_mat(mat_lst):
    result = mat_lst[0].copy()
    if len(mat_lst) > 1:
        for mat in mat_lst[1:]:
            result += mat
    return result

# adaboost主算法
def adaboost(x_mat, y_mat, max_classifier_num=30):
    weak_classifiers = []
    weak_classifiers_result = []
    m, n = x_mat.shape
    w = np.matrix(np.array([1 / n] * n).reshape(1, n))
    for i in range(max_classifier_num):
        print('------------------------------')
        print('Round {}'.format(i + 1))
        print('The normalized w: {}'.format(str(list(np.round(np.array(w), 3)[0]))))
        weak_classifier, accuracy, weak_classifier_result = build_stump(x_mat, y_mat, w)
        weak_classifiers.append(weak_classifier)
        err = 1 - accuracy
        alpha = 0.5 * np.log((1 - err)/err)
        weak_classifiers_result.append(alpha * weak_classifier_result)
        print('The Classifier:')
        dim, thresh_val, direction = weak_classifier
        classifier_lst = sorted(list(set(np.array(x_mat)[dim])) + [thresh_val], reverse=not direction)
        classifier_lst[classifier_lst.index(thresh_val)] = '|'
        classifier_lst = map(str, classifier_lst)
        classifier_str = ' '.join(classifier_lst)
        if classifier_str[-1] == '|':
            classifier_str += ' None'
        if classifier_str[0] == '|':
            classifier_str = ['None '] + classifier_str
        print(classifier_str)
        print('\nThe error is {:.03f}'.format(err))
        print('The alpha is {:.03f}'.format(alpha))
        print('The weak classifier\'s prediction result: \n{}'.format(str(np.sign(weak_classifier_result))))
        print('The whole bag of classifications\'s result: \n{}'.format(str(np.sign(sum_mat(weak_classifiers_result)))))

        if np.average(np.multiply(sum_mat(weak_classifiers_result), y_mat) > 0) == 1:
            print('------------------------------')
            return weak_classifiers
        
        w = np.multiply(w, np.exp(- alpha * np.multiply(np.sign(weak_classifier_result), y_mat)))
        w = w / np.sum(w)
    print('------------------------------')
    return weak_classifiers


classifiers = adaboost(x_mat, y_mat, 10)

------------------------------
Round 1
The normalized w: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
The Classifier:
1 0 | None

The error is 0.200
The alpha is 0.693
The weak classifier's prediction result: 
[[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]]
The whole bag of classifications's result: 
[[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]]
------------------------------
Round 2
The normalized w: [0.062, 0.062, 0.062, 0.062, 0.062, 0.062, 0.25, 0.25, 0.062, 0.062]
The Classifier:
3 2 | 1

The error is 0.188
The alpha is 0.733
The weak classifier's prediction result: 
[[ 1. -1. -1.  1. -1.  1.  1.  1. -1. -1.]]
The whole bag of classifications's result: 
[[ 1. -1. -1.  1. -1.  1.  1.  1. -1. -1.]]
------------------------------
Round 3
The normalized w: [0.167, 0.038, 0.038, 0.167, 0.038, 0.167, 0.154, 0.154, 0.038, 0.038]
The Classifier:
3 2 | 1

The error is 0.269
The alpha is 0.499
The weak classifier's prediction result: 
[[-1.  1. -1. -1. -1. -1. -1.  1.  1.  1.]]
The whole ba

In [18]:
sorted(list(set(np.array(x_mat)[2])) + [2.5], reverse=not True)

[1, 2, 2.5, 3]

In [23]:
set(np.array(x_mat)[2])

{1, 2, 3}

In [ ]:
# 单层树桩分类器
def stump_classify(mat, dim, thresh_val, ascending):
    result = np.ones((1, mat.shape[1]))
    x = mat[dim]
    for i in range(len(x)):
        if ascending is True:
            result[mat[dim, :] <= thresh_val] = -1
        else:
            result[mat[dim, :] > thresh_val] = -1
    return result

# 找到最优树桩（不允许预测结果全部取1和全部取-1）
def build_stump(x_mat, y_mat, w):
    m, n = x_mat.shape
    best_stump = []
    best_acc = -1
    best_result = np.matrix([0])
    w_sum = np.sum(w)
    for dim in range(m):
        unique_vals = sorted(set(np.array(x_mat)[dim]))
        for val_floor, val_ceil in zip(unique_vals[:-1], unique_vals[1:]): # 这里不能全部取负
            ascending = True
            thresh_val = (val_floor + val_ceil) / 2
            pred_result = stump_classify(x_mat, dim, thresh_val, ascending=True)
            accuracy = np.sum(np.multiply((pred_result == y_mat), w))
            if accuracy < w_sum / 2:
                ascending = False
                accuracy = w_sum - accuracy
                pred_result = - pred_result
            if accuracy > best_acc:
                best_stump = [dim, thresh_val, ascending]
                best_acc = accuracy
                best_result = pred_result
    return best_stump, best_acc, best_result

# 计算矩阵之和
def sum_mat(mat_lst):
    result = mat_lst[0].copy()
    if len(mat_lst) > 1:
        for mat in mat_lst[1:]:
            result += mat
    return result

# adaboost主算法
def adaboost(x_mat, y_mat, max_classifier_num=30):
    weak_classifiers = []
    weak_classifiers_result = []
    m, n = x_mat.shape
    w = np.matrix(np.array([1 / n] * n).reshape(1, n))
    for i in range(max_classifier_num):
        weak_classifier, accuracy, weak_classifier_result = build_stump(x_mat, y_mat, w)
        weak_classifiers.append(weak_classifier)
        err = 1 - accuracy
        alpha = 0.5 * np.log((1 - err)/err)
        weak_classifiers_result.append(alpha * weak_classifier_result)

        if np.average(np.multiply(sum_mat(weak_classifiers_result), y_mat) > 0) == 1:
            return weak_classifiers
        
        w = np.multiply(w, np.exp(- alpha * np.multiply(np.sign(weak_classifier_result), y_mat)))
        w = w / np.sum(w)
    return weak_classifiers

len(adaboost(x_mat, y_mat, 100))